In [2]:
!pip install phe > /dev/null

In [4]:
# ----------------------------
# Imports and small helpers
# ----------------------------
from phe import paillier    # Paillier implementation (python-paillier)
import numpy as np
import time

def pretty(title, x):
    """Small helper to print titles and values nicely."""
    print("\n=== {} ===".format(title))
    print(x)

# ----------------------------
# 1) Key generation
# ----------------------------
# generate_paillier_keypair creates a (public_key, private_key) tuple.
# n_length controls RSA modulus size; 2048 is common and reasonably secure for demos.
public_key, private_key = paillier.generate_paillier_keypair(n_length=2048)
print("Generated Paillier keypair (2048-bit).")

# ----------------------------
# 2) Simple scalar demo (integers)
# ----------------------------
# Plaintext integers to encrypt
m1 = 42
m2 = 17

# Encrypt using the public key: public_key.encrypt(plaintext)
enc1 = public_key.encrypt(m1)
enc2 = public_key.encrypt(m2)

# Homomorphic addition of ciphertexts:
# enc_sum = E(m1) + E(m2)  => decrypt(enc_sum) == m1 + m2
enc_sum = enc1 + enc2

# Homomorphic scalar multiplication:
# enc_scaled = E(m1) * k  => decrypt(enc_scaled) == k * m1
k = 7
enc_scaled = enc1 * k

# Decrypt results using the private key
dec_sum = private_key.decrypt(enc_sum)
dec_scaled = private_key.decrypt(enc_scaled)

pretty("m1 (plaintext)", m1)
pretty("m2 (plaintext)", m2)
pretty("Decrypted m1 + m2", dec_sum)
pretty(f"Decrypted {k} * m1", dec_scaled)

# ----------------------------
# 3) Vector example: encrypt element-wise & compute encrypted sum
# ----------------------------
# Paillier does not pack many values into one ciphertext (no SIMD like CKKS),
# so we encrypt each element separately in this simple demo.
vec = np.array([5, 10, 15, 20], dtype=int)
pretty("Plain vector", vec)

# Encrypt each element (element-wise encryption)
enc_vec = [public_key.encrypt(int(x)) for x in vec]

# Homomorphically add all ciphertexts to compute encrypted sum:
enc_vec_sum = enc_vec[0]
for c in enc_vec[1:]:
    enc_vec_sum += c

# Decrypt the encrypted sum and compare to plaintext sum
dec_vec_sum = private_key.decrypt(enc_vec_sum)
pretty("Decrypted sum of vector (homomorphic)", dec_vec_sum)
pretty("Plain sum", int(np.sum(vec)))

# ----------------------------
# 4) Privacy-preserving mean example
# ----------------------------
# Scenario: server has encrypted elements enc_vec and computes the encrypted sum.
# Server sends encrypted sum back; only the private key holder can decrypt and compute the mean.
n = len(vec)
enc_sum = enc_vec_sum      # server-side result (ciphertext)
decrypted_sum = private_key.decrypt(enc_sum)
mean = decrypted_sum / n   # compute mean in the clear after decryption
pretty(f"Decrypted mean (sum/{n})", mean)

# ----------------------------
# 5) Working with floats (fixed-point encoding)
# ----------------------------
# Paillier works natively with integers. To handle floats:
#  - Multiply floats by a scaling factor (e.g., scale = 10**6) -> convert to integers
#  - Encrypt and operate on those integers
#  - After decryption, divide by the scale to recover approximate floats
# Example:
scale = 10**6
float_vals = [1.234567, 2.5, -0.333333]
pretty("Plain floats", float_vals)
scaled_ints = [int(round(x * scale)) for x in float_vals]
pretty("Scaled to integers (fixed-point)", scaled_ints)

# Encrypt scaled integers and compute encrypted sum
enc_scaled_vals = [public_key.encrypt(x) for x in scaled_ints]
enc_scaled_sum = enc_scaled_vals[0]
for c in enc_scaled_vals[1:]:
    enc_scaled_sum += c

# Decrypt and unscale back to float
dec_scaled_sum = private_key.decrypt(enc_scaled_sum)
dec_float_sum = dec_scaled_sum / scale
pretty("Decrypted float sum (after unscale)", dec_float_sum)
pretty("Plain float sum", sum(float_vals))

# ----------------------------
# 6) Quick performance check
# ----------------------------
N = 200
sample_vals = np.random.randint(0, 1000, size=N)
t0 = time.time()
encs = [public_key.encrypt(int(v)) for v in sample_vals]
enc_sum2 = encs[0]
for c in encs[1:]:
    enc_sum2 += c
t1 = time.time()
dec_sum2 = private_key.decrypt(enc_sum2)
t2 = time.time()

print(f"\nTime to encrypt {N} ints + sum ciphertexts: {t1-t0:.3f}s; decrypt: {t2-t1:.3f}s")

# ----------------------------
# 7) Notes & limitations (printed for user's attention)
# ----------------------------
print("\nNotes:")
print("- Paillier supports only additive homomorphism: you can add ciphertexts and multiply ciphertexts by integer scalars.")
print("- There is NO native ciphertext*ciphertext multiplication (no multiplicative homomorphism).")
print("- For floats, use fixed-point scaling as shown; be mindful of modulus size and overflow for large scales or many operations.")
print("- Paillier encrypts each number separately (no packing). For large vectors, think about performance and storage tradeoffs.")

Generated Paillier keypair (2048-bit).

=== m1 (plaintext) ===
42

=== m2 (plaintext) ===
17

=== Decrypted m1 + m2 ===
59

=== Decrypted 7 * m1 ===
294

=== Plain vector ===
[ 5 10 15 20]

=== Decrypted sum of vector (homomorphic) ===
50

=== Plain sum ===
50

=== Decrypted mean (sum/4) ===
12.5

=== Plain floats ===
[1.234567, 2.5, -0.333333]

=== Scaled to integers (fixed-point) ===
[1234567, 2500000, -333333]

=== Decrypted float sum (after unscale) ===
3.401234

=== Plain float sum ===
3.401234

Time to encrypt 200 ints + sum ciphertexts: 19.860s; decrypt: 0.026s

Notes:
- Paillier supports only additive homomorphism: you can add ciphertexts and multiply ciphertexts by integer scalars.
- There is NO native ciphertext*ciphertext multiplication (no multiplicative homomorphism).
- For floats, use fixed-point scaling as shown; be mindful of modulus size and overflow for large scales or many operations.
- Paillier encrypts each number separately (no packing). For large vectors, think a